In [ ]:
#this code is used to load the Data file CSV into My system and storing it into dataframe
#chunk wise loading to save memory and time windows 1932 only work here to acoid errors idk why m-swasth issue in data
import time
import pandas as pd

chunk_size = 60000

# Initialize an empty list to store the chunks
processed_data = []

# Read the specified rows in chunks from the CSV file
for chunk in pd.read_csv(r"C:\Users\satvi\OneDrive\Desktop\m-swasth\2024\feb\all_partner_recon_report\all_partner_recon_report.csv", sep = '|', encoding = 'windows-1252',
                         encoding_errors='ignore',on_bad_lines ='skip', chunksize = chunk_size):
    print("added chunk")
    processed_data.append(chunk)

# Concatenate the chunks into a single DataFrame
processed_df_1 = pd.concat(processed_data)
headers = pd.read_csv(r"C:\Users\satvi\OneDrive\Desktop\m-swasth\2024\feb\all_partner_recon_report\all_partner_recon_report.csv", nrows = 1, sep = '|')
processed_df_1.columns = headers.columns
print('Processing data...')

In [ ]:
#converting them into date time format here yyyy=mm-dd

# Assuming processed_df_1 is your DataFrame
processed_df_1['Subscription Date'] = pd.to_datetime(processed_df_1['Subscription Date'])
processed_df_1['Expiry Date'] = pd.to_datetime(processed_df_1['Expiry Date'])
processed_df_1['Payment Date'] = pd.to_datetime(processed_df_1['Payment Date'])



In [ ]:
#keeping only the required columns here
columns_to_keep = ['Channel', 'Policy #', 'Member Id', 'Loan #', 'Subscription Date', 'Expiry Date', 'Branch Name', 'Branch Id', 'Payment Date']
processed_df_1 = processed_df_1[columns_to_keep]


In [ ]:
#creating a pivot for each channel to know the count of policy
#WAY 1
# Assuming your DataFrame is named processed_df_1
pivot_df = processed_df_1.groupby(['Channel', 'Policy #', 'Payment Date']).size().reset_index(name='Count')

# Pivot the data
pivot_table = pivot_df.pivot_table(index=['Channel', 'Policy #', 'Count'], values='Payment Date', aggfunc=lambda x: ', '.join(x.dt.strftime('%d %B %Y')))

print(pivot_table)


In [ ]:
#WAY 2

import pandas as pd

# Assuming you already have your data in a DataFrame called 'processed_df_1'

# Create the new DataFrame
result = (processed_df_1.groupby(['Member Id', 'Channel'])
          .size()
          .to_frame(name='count')
          .reset_index()
          .merge(processed_df_1[['Member Id', 'Channel', 'Payment Date']], how='left', on=['Member Id', 'Channel'])
          .groupby(['Member Id', 'Channel'])['Payment Date']
          .agg(lambda x: ', '.join(x.dt.strftime('%d %b %Y')))
          .reset_index())

# Display the result
print(result)

#this will create a table where i can see each member id from each channel adn there payments date
#id channel payment_date
#001  XYZ   21 jan 2022, 21 jan 2023, 21 jan 2024






In [ ]:
import pandas as pd

# Assuming 'result' is the DataFrame containing the grouped data as shown in your example

# Split the rows with multiple payment dates into separate rows
result_expanded = result.assign(Payment_Date=result['Payment Date'].str.split(', ')).explode('Payment_Date')

# Display the expanded result
print(result_expanded[['Member Id', 'Channel', 'Payment_Date']])


#now we will exploed

In [ ]:
import pandas as pd

# Assuming 'result_expanded' is the DataFrame containing the expanded data after exploding the rows

# Count the occurrences of each Member Id
member_id_counts = result_expanded['Member Id'].value_counts()

# Filter Member Ids that have repeated more than once
repeated_member_ids = member_id_counts[member_id_counts > 1].index

# Filter and store the information of repeating Member Ids into temp_data
temp_data = result_expanded[result_expanded['Member Id'].isin(repeated_member_ids)]

# Display temp_data
print(temp_data)


#now we will have the users who are repeating here and there payment dates here also


In [ ]:
import pandas as pd
import numpy as np

# Assuming 'temp_data' is the DataFrame containing the relevant data

# Convert 'Payment_Date' column to datetime
temp_data['Payment_Date'] = pd.to_datetime(temp_data['Payment_Date'])

# Calculate the time intervals between purchases for each Member Id
temp_data['Time_Interval'] = temp_data.groupby('Member Id')['Payment_Date'].diff().dt.days

# Categorize the time intervals into different categories
conditions = [
    temp_data['Time_Interval'].isnull(),  # First purchase (NaN)
    temp_data['Time_Interval'] < 5,
    (temp_data['Time_Interval'] >= 5) & (temp_data['Time_Interval'] <= 30),
    (temp_data['Time_Interval'] > 30) & (temp_data['Time_Interval'] <= 90),
    (temp_data['Time_Interval'] > 90) & (temp_data['Time_Interval'] <= 200),
    (temp_data['Time_Interval'] > 200) & (temp_data['Time_Interval'] <= 365),
    temp_data['Time_Interval'] > 365
]

categories = ['First Purchase', 'Less than 5 days', '5-30 days', '31-90 days', '91-200 days', '201-365 days', 'More than 365 days']

# Create a new column 'Time_Category' based on the time interval categories
temp_data['Time_Category'] = np.select(conditions, categories, default='Unknown')

# Count the number of occurrences for each time interval category
time_interval_counts = temp_data.groupby('Time_Category').size()

# Display the counts for each time interval category
for category, count in time_interval_counts.items():
    print(f"{category} - {count}")


#now we will be having the users here under certain condition


conditions are
201-365 days - 388748
31-90 days - 39453
5-30 days - 32157
91-200 days - 131405
First Purchase - 3202493
Less than 5 days - 4061519
More than 365 days - 708576

In [ ]:
#following the same conditions we will be seeing how conditions are met seprate for each channel


import pandas as pd
import numpy as np

# Assuming 'temp_data' is the DataFrame containing the relevant data

# Convert 'Payment_Date' column to datetime
temp_data['Payment_Date'] = pd.to_datetime(temp_data['Payment_Date'])

# Calculate the time intervals between purchases for each Member Id within each Channel
temp_data['Time_Interval'] = temp_data.groupby(['Channel', 'Member Id'])['Payment_Date'].diff().dt.days

# Categorize the time intervals into different categories for each Channel
conditions = [
    temp_data['Time_Interval'].isnull(),  # First purchase (NaN)
    temp_data['Time_Interval'] < 5,
    (temp_data['Time_Interval'] >= 5) & (temp_data['Time_Interval'] <= 30),
    (temp_data['Time_Interval'] > 30) & (temp_data['Time_Interval'] <= 90),
    (temp_data['Time_Interval'] > 90) & (temp_data['Time_Interval'] <= 200),
    (temp_data['Time_Interval'] > 200) & (temp_data['Time_Interval'] <= 365),
    temp_data['Time_Interval'] > 365
]

categories = ['First Purchase', 'Less than 5 days', '5-30 days', '31-90 days', '91-200 days', '201-365 days', 'More than 365 days']

# Create a new column 'Time_Category' based on the time interval categories
temp_data['Time_Category'] = np.select(conditions, categories, default='Unknown')

# Count the number of occurrences for each time interval category for each Channel
time_interval_counts_by_channel = temp_data.groupby(['Channel', 'Time_Category']).size()

# Display the counts for each time interval category for each Channel
print(time_interval_counts_by_channel)


In [ ]:
#just to see how many users are coming for the first time here
temp_data.drop(columns=['Payment Date'], inplace=True)

# Display the updated temp_data after dropping the 'Payment Date' column
print(temp_data)
